# Assignment 2

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from fuzzywuzzy import fuzz

In [ ]:
hospital1 = pd.read_excel('./hospital1.xlsx', index_col=None)
hospital1.head()

In [ ]:
hospital2 = pd.read_excel('./hospital2.xlsx', index_col=None)
hospital2.head()

### 1. Data Exploration

1.1. Hospital 1. Check If 'patient ID' and 'patient ID.1' are the same  

In [ ]:
different_rows = hospital1[hospital1['patient ID'] != hospital1['patient ID.1']]

if not different_rows.empty:
    print("The following rows have different values between 'patient ID' and 'patient ID.1':")
    print(different_rows)
    print("Total different rows: " + str(len(different_rows)))
else:
    print("There are no rows with different values between 'patient ID' and 'patient ID.1'.")

1.1.1 Check date of first symptoms and BASVURUTARIHI (means application date in Turkish)

In [ ]:
different_date_rows = hospital1[hospital1['date_of_first_symptoms'] != hospital1['BASVURUTARIHI']]

if not different_date_rows.empty:
    print("The following rows have different values between 'date_of_first_symptoms' and 'BASVURUTARIHI':")
    print(different_date_rows)
    print("Total different date rows: " + str(len(different_date_rows)))
else:
    print("There are no rows with different values between 'date_of_first_symptoms' and 'BASVURUTARIHI'.")

1.2 Hospital 2. Check if patients are admitted multiple times (same patient id different admission id) 

In [ ]:
# Find patient IDs with multiple admission IDs
multiple_admissions = hospital2.groupby('patient_id')['admission_id'].nunique()

# Filter to find patient IDs with more than one unique admission ID
multiple_admissions = multiple_admissions[multiple_admissions > 1]

if not multiple_admissions.empty:
    print("The following patient IDs have multiple admission IDs:")
    print(multiple_admissions)
    print('Total multiple admissions for one patient id, rows: ' + str(len(multiple_admissions))) 
else:
    print("No patient has multiple admission IDs.")

1.3 Together 
    - Similar Rows
    - Same rows in both datasets with different results  

In [ ]:
# Define a similarity threshold
similarity_threshold = 100

# Find similar columns
similar_columns = []
for col1 in hospital1.columns:
    for col2 in hospital2.columns:
        similarity_score = fuzz.ratio(col1, col2)
        if similarity_score >= similarity_threshold:
            similar_columns.append((col1, col2, similarity_score))

# Display results
if similar_columns:
    print("Columns with similar meanings:")
    for col1, col2, score in similar_columns:
        print(f"'{col1}' in hospital1 and '{col2}' in hospital2 with similarity score: {score}")
    print('Total columns with similar meanings: ' + str(len(similar_columns)))
else:
    print("No columns with similar meanings found.")

In [ ]:
# Define a similarity threshold
similarity_threshold = 64

# Find similar columns
similar_columns = []
for col1 in hospital1.columns:
    for col2 in hospital2.columns:
        similarity_score = fuzz.ratio(col1, col2)
        if similarity_score >= similarity_threshold and similarity_score != 100:
            similar_columns.append((col1, col2, similarity_score))

# Display results
if similar_columns:
    print("Columns with similar meanings:")
    for col1, col2, score in similar_columns:
        print(f"'{col1}' in hospital1 and '{col2}' in hospital2 with similarity score: {score}")
    print('Total columns with similar meanings: ' + str(len(similar_columns)))
else:
    print("No columns with similar meanings found.")


1.4 Check all columns  
    - Missing values  
    - Null values  
    - Inconsistencies  
    - (Check slides for more) 

In [ ]:
# Count missing values in each column
missing_values = hospital1.isnull().sum()
print("Missing values per column:")
print(missing_values)

In [ ]:
# Check if each column has any missing values
missing_columns = hospital1.isnull().any()
print("Columns with missing values:")
print(missing_columns[missing_columns])

In [ ]:
# Find rows with any missing values
rows_with_missing = hospital1[hospital1.isnull().any(axis=1)]
print("Rows with missing values:")
print(rows_with_missing)

In [ ]:
hospital1.info()

In [ ]:
plt.figure(figsize=(20, 10))

# Visualize missing values
sns.heatmap(hospital1.isnull(), cbar=False, cmap="viridis")
plt.show()

In [ ]:
# Count missing values in each column
missing_values = hospital2.isnull().sum()
print("Missing values per column:")
print(missing_values)

In [ ]:
# Check if each column has any missing values
missing_columns = hospital2.isnull().any()
print("Columns with missing values:")
print(missing_columns[missing_columns])

In [ ]:
# Find rows with any missing values
rows_with_missing = hospital2[hospital2.isnull().any(axis=1)]
print("Rows with missing values:")
print(rows_with_missing)

In [ ]:
hospital2.info()

In [ ]:
plt.figure(figsize=(20, 10))

# Visualize missing values
sns.heatmap(hospital2.isnull(), cbar=False, cmap="viridis")
plt.show()


1.5 Create visualization  

In [ ]:
# Count the number of positive and negative PCR results
pcr_counts = hospital1['PCR_result'].value_counts()
print(pcr_counts)

# Create a bar plot to visualize PCR results
plt.figure(figsize=(8, 6))
pcr_counts.plot(kind='bar', color=['blue', 'red'])

# Add titles and labels
plt.title('PCR Test Results', fontsize=14)
plt.xlabel('PCR Result', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)  # Rotate the x-axis labels for readability

# Show the plot
plt.show()

In [ ]:
# Count the number of positive and negative PCR results
pcr_counts = hospital2['PCR_result'].value_counts()
print(pcr_counts)

# Create a bar plot to visualize PCR results
plt.figure(figsize=(8, 6))
pcr_counts.plot(kind='bar', color=['blue', 'red'])

# Add titles and labels
plt.title('PCR Test Results', fontsize=14)
plt.xlabel('PCR Result', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)  # Rotate the x-axis labels for readability

# Show the plot
plt.show()

In [ ]:
# Example: Count plot for the nationality column
plt.figure(figsize=(22, 2))
sns.countplot(x='nationality', data=hospital1)
plt.title('Distribution of Nationalities')
plt.xticks(rotation=45, ha='right')  # Rotate labels for better readability
plt.xlabel('Nationality')
plt.ylabel('Count')
plt.show()

In [ ]:
# Example: Horizontal bar plot for nationality distribution
hospital1_filtered = hospital1[hospital1['nationality'] != 'T.C.']
nationality_counts = hospital1_filtered['nationality'].value_counts()

plt.figure(figsize=(10, 18))
sns.barplot(x=nationality_counts.values, y=nationality_counts.index)
plt.title('Nationality Distribution (without T.C.)')
plt.xlabel('Count')
plt.ylabel('Nationality')
plt.show()

In [ ]:
# Box plot to visualize oxygen saturation
plt.figure(figsize=(18, 4)) 
sns.boxplot(x=hospital1['oxygen_saturation'], color='lightgreen')
plt.title('Box Plot of Oxygen Saturation')
plt.xlabel('Oxygen Saturation (%)')
plt.show()

Ranges for Oxygen Saturation:
- 95% to 100%: Normal
- 90% to 94%: Mildly low (monitor, consult if persistent)
- Below 90%: Hypoxemia (medical attention recommended)
- Below 85%: Severe hypoxemia (urgent medical intervention required)

In [ ]:
num_columns = hospital1.select_dtypes(include='number').columns  # Get numerical columns
cat_columns = hospital1.select_dtypes(include='object').columns   # Get categorical columns

print("numerical columns: " + str(len(num_columns)))
print("categorical columns: " + str(len(cat_columns)))

# Create plots for categorical columns
plt.figure(figsize=(15, 10))
for i, col in enumerate(cat_columns):
    plt.subplot(2, len(cat_columns) // 2 + 1, i + 1)
    sns.countplot(x=hospital1[col])  # Count plot for categorical variables
    plt.title(f'Count of {col}')
    plt.xticks(rotation=45)
    plt.tight_layout()

plt.show()

In [ ]:
# Set up the plotting space for multiple plots
num_columns = hospital1.select_dtypes(include='number').columns  # Get numerical columns

# Create plots for numerical columns
plt.figure(figsize=(40, 20))
for i, col in enumerate(num_columns):
    plt.subplot(4, len(num_columns) // 4 + 1, i + 1)  # Set a grid for subplots
    sns.histplot(hospital1[col], kde=True, bins=20, color='skyblue')  # You can change this to other plots (e.g., boxplot)
    plt.title(f'Distribution of {col}')
    plt.tight_layout()  # To avoid overlapping subplots

plt.show()

In [ ]:
# Set up the plotting space for multiple plots
num_columns = hospital1.select_dtypes(include='number').columns  # Get numerical columns

# Create plots for numerical columns
plt.figure(figsize=(40, 20))
for i, col in enumerate(num_columns):
    plt.subplot(4, len(num_columns) // 4 + 1, i + 1)  # Set a grid for subplots
    sns.boxplot(x=hospital1[col], color='lightgreen')
    plt.title(f'Box Plot of {col}')
    plt.tight_layout()  # To avoid overlapping subplots

plt.show()

In [ ]:
num_columns_2 = hospital2.select_dtypes(include='number').columns  # Get numerical columns
cat_columns_2 = hospital2.select_dtypes(include='object').columns   # Get categorical columns

print("numerical columns: " + str(len(num_columns_2)))
print("categorical columns: " + str(len(cat_columns_2)))

# Create plots for categorical columns
plt.figure(figsize=(15, 10))
for i, col in enumerate(cat_columns_2):
    plt.subplot(2, len(cat_columns_2) // 2 + 1, i + 1)
    sns.countplot(x=hospital2[col])  # Count plot for categorical variables
    plt.title(f'Count of {col}')
    plt.xticks(rotation=45)
    plt.tight_layout()

plt.show()

In [ ]:
# Set up the plotting space for multiple plots
num_columns_2 = hospital2.select_dtypes(include='number').columns  # Get numerical columns

# Create plots for numerical columns
plt.figure(figsize=(40, 20))
for i, col in enumerate(num_columns_2):
    plt.subplot(4, len(num_columns_2) // 4 + 1, i + 1)  # Set a grid for subplots
    sns.histplot(hospital2[col], kde=True, bins=20, color='skyblue')  # You can change this to other plots (e.g., boxplot)
    plt.title(f'Distribution of {col}')
    plt.tight_layout()  # To avoid overlapping subplots

plt.show()

In [ ]:
# Set up the plotting space for multiple plots
num_columns_2 = hospital2.select_dtypes(include='number').columns  # Get numerical columns

# Create plots for numerical columns
plt.figure(figsize=(40, 20))
for i, col in enumerate(num_columns_2):
    plt.subplot(4, len(num_columns_2) // 4 + 1, i + 1)  # Set a grid for subplots
    sns.boxplot(x=hospital2[col], color='lightgreen')
    plt.title(f'Box Plot of {col}')
    plt.tight_layout()  # To avoid overlapping subplots

plt.show()

### Descriptive analysis

Columns with mising values:

fever_temperature                
oxygen_saturation                
chronic_kidney_disease           
obesity                          
liver_disease                    
asplenia                         
chronic_neurological_disorder    
chronic_hematologic_disease      
AIDS_HIV                         
diabetes_mellitus_type_1         
diabetes_mellitus_type_2         
rheumatologic_disorder           
dementia                         
PCR_result                       

In [ ]:
mean_fever_temperature = hospital1['fever_temperature'].mean()
print("Mean of the column fever_temperature: ", mean_fever_temperature)

median_fever_temperature = hospital1['fever_temperature'].median()
print("Median of the column fever_temperature: ", median_fever_temperature)

min_fever_temperature = hospital1['fever_temperature'].min()
print("Min of the column fever_temperature: ", min_fever_temperature)

max_fever_temperature = hospital1['fever_temperature'].max()
print("Max of the column fever_temperature: ", max_fever_temperature)


In [ ]:
mean_oxygen_saturation = hospital1['oxygen_saturation'].mean()
print("Mean of the column oxygen_saturation: ", mean_oxygen_saturation)

median_oxygen_saturation = hospital1['oxygen_saturation'].median()
print("Median of the column oxygen_saturation: ", median_oxygen_saturation)

min_oxygen_saturation = hospital1['oxygen_saturation'].min()
print("Min of the column oxygen_saturation: ", min_oxygen_saturation)

max_oxygen_saturation = hospital1['oxygen_saturation'].max()
print("Max of the column oxygen_saturation: ", max_oxygen_saturation)


We can detect **outliers in oxygen_saturation** column.